In [1]:
import numpy as np
import pandas as pd
import zipfile
from IPython.display import display, HTML

# Processing votes

In [24]:
def interpret_position(positions):
    r = positions.copy()
    r = r.apply(lambda x : 1 if x == "Yes" else ( -1 if x == "No" else 0 ))
    
    return r

USEFUL_COLUMNS = ['id','bill.number','congress','member_id','question','position', 'timestamp', 'bill.bill_id']

def process_vote_file(file_path, since_congress=115):
    print("Processing vote data for {file}".format(file=file_path), end='\r')
    ifile = ZIPPED_VOTES.open(file_path)
    
    # Read data
    df = pd.read_csv(ifile)

    # We drop entries without bill number as they lead to invalid bill IDs
    df = df[df['bill.number'].notna()]
    
    # We drop entries without roll call value
    df = df[df['roll_call'].notna()]
    
    # As there can be several votes per bill we need to build a unique ID
    df['id'] = df['bill.bill_id'].map(str) + "-" + df['roll_call'].map(str)

    # Some bill IDs and roll call numbers are poorly filled, leading to duplicates
    df = df.drop_duplicates('id')
    
    # Build timestamps
    df['timestamp'] = pd.to_datetime(df.date.map(str) + " " + df.time.map(str))
    
    # Keep only useful columns
    df = df[USEFUL_COLUMNS]
    
    # Keep only relevant congresses
    df = df[df['congress'] >= since_congress]
    
    # Convert position to numeric
    df['position'] = interpret_position(df['position'])
    
    return df

NOMINATION_VOTE = "On the Nomination"
MOTION_VOTE = "On the Motion"
AMENDMENT_VOTE = "On the Amendment"
MOTION_TO_TABLE_VOTE = "On the Motion to Table"
CLOTURE_MOTION_VOTE = "On the Cloture Motion"
BILL_PASSAGE = "On Passage of the Bill"

def build_vote_matrix(questions, data):
    """
    Given a list of questions, build a vote matrix.
    Index are senators IDs, columns are vote IDs and values are the respective positions
    of the senators with respect to the question (Yes, No, blank or NaN if the senator
    did not take part in the vote)
    
    Data should contain the following columns:
        - senators ID (member_id)
        - vote ID (id)
        - position (position)
    """    
    
    vote_matrix = data 
    
    if len(questions) > 0:
        vote_matrix = data[data['question'].isin(questions)]
        
    vote_matrix = vote_matrix.pivot(columns='id', index='member_id', values='position')
    
    return vote_matrix

## Read and compile votes per senators in a matrix

In [25]:
ZIPPED_VOTES = zipfile.ZipFile('data/votes/votes.zip')
vote_data_files = [x.filename for x in ZIPPED_VOTES.infolist() if x.filename.startswith('votes_')]

df_list = []

for filepath in vote_data_files:
    vote_data = process_vote_file(filepath)
    df_list.append(vote_data)
    
raw_votes = pd.concat(df_list)
raw_votes.to_pickle("data/processed/processed_vote_data.pickle")

In [6]:
vote_matrix = build_vote_matrix([BILL_PASSAGE, MOTION_VOTE], raw_votes).fillna(0)
display(vote_matrix)
vote_matrix.to_pickle("data/processed/processed_votes.pickle")
np.save("data/processed/processed_senators_ids.npy", vote_matrix.index.values)

id,hconres71-115-224,hconres71-115-227,hconres71-115-231,hconres71-115-234,hjres38-115-43,hr1-115-285,hr1-115-286,hr1-115-287,hr1-115-288,hr1-115-289,...,sconres3-115-22,sconres3-115-23,sconres3-115-24,sconres3-115-25,sconres3-115-4,sconres3-115-5,sconres3-115-6,sconres3-115-7,sconres3-115-8,sconres3-115-9
member_id,,,,,,,,,,,,,,,,,,,,,
A000360,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
B000575,0.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,1.0
B000944,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,-1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0
B001135,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0
B001230,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,-1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0
B001236,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
B001261,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
B001267,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,-1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0
B001277,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,-1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0


# Processing bills

In [43]:
bill_ids = pd.DataFrame(raw_votes['bill.bill_id'].drop_duplicates()).set_index('bill.bill_id')

df_bills = pd.read_csv("data/bills/active_bills.csv")
df_bills = df_bills[['bill_id','summary','summary_short']]

bill_descr = bill_ids.merge(df_bills, left_index=True, right_on='bill_id', how='left').set_index('bill_id')
bill_descr.to_csv("data/processed/processed_bill_descriptions.csv")

# Processing senators

In [11]:
pd.read_csv("data/senate_members/senate_members_115.csv")[['id','first_name', 'last_name','party']]

,id,first_name,last_name,party
0,A000360,Lamar,Alexander,R
1,B001230,Tammy,Baldwin,D
2,B001261,John,Barrasso,R
3,B001267,Michael,Bennet,D
4,B001277,Richard,Blumenthal,D
5,B000575,Roy,Blunt,R
6,B001288,Cory,Booker,D
7,B001236,John,Boozman,R
8,B000944,Sherrod,Brown,D
9,B001135,Richard,Burr,R


In [12]:
raw_senators = pd.read_csv("data/senate_members/senate_members_115.csv")[['id', 'party', 'first_name', 'last_name']].set_index('id')
raw_senators['name'] = raw_senators['first_name'].map(str) + " " + raw_senators['last_name'].map(str)
senators = raw_senators.loc[vote_matrix.index]
senators.to_pickle("data/processed/processed_senators.pickle")

# Saving party info
np.save("data/processed/processed_labels.npy", senators['party'].values)
labels = senators['party'].values

# Saving numerical values for party
dict_mapping = {'R':0.,'D':0.12,'I':0.223}
labels = np.vectorize(dict_mapping.get)(labels)
np.save("data/processed/processed_labels_numerical.npy", labels)